# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
# Load the environment
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Load the environment variable
PRICE_DATA = os.getenv("PRICE_DATA")
# import shutil
# if os.path.exists(PRICE_DATA):
#     shutil.rmtree(PRICE_DATA)

# Recursively find all .parquet files
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker") # read all parquet files and set index to ticker, this returns a Dask DataFrame



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [18]:
# Write your code below.

dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1),
        Returns = x['Close'] / x['Close'].shift(1) - 1,
        Hi_Lo_range = x['High'] - x['Low']
    )
)


/var/folders/2t/_qccqblj6sbg3zx9bhx_g6tm0000gn/T/ipykernel_34303/574656064.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(


In [19]:
dd_feat

Dask DataFrame Structure:
                          Date     Open     High      Low    Close Adj Close   Volume  source   Year Close_lag_1 Adj_Close_lag_1  Returns Hi_Lo_range
npartitions=60                                                                                                                                       
A               datetime64[ns]  float64  float64  float64  float64   float64  float64  string  int32     float64         float64  float64     float64
ACB                        ...      ...      ...      ...      ...       ...      ...     ...    ...         ...             ...      ...         ...
...                        ...      ...      ...      ...      ...       ...      ...     ...    ...         ...             ...      ...         ...
ZEUS                       ...      ...      ...      ...      ...       ...      ...     ...    ...         ...             ...      ...         ...
ZEUS                       ...      ...      ...      ...      ...       ...      ...     ...    ...         ...             ...      ...         ...
Dask Name: groupbyapply, 3 expressions
Expr=GroupByApply(frame=SetIndex(frame=ReadParquetFSSpec(2b8caaf), _other='ticker', options={}), observed=False, group_keys=False, func=<function <lambda> at 0x13cfb6820>, meta=<no_default>, args=(), kwargs={})

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.

df_feat = dd_feat.compute() # Convert Dask DataFrame to Pandas DataFrame

In [21]:
dd_feat.compute().head()

Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      1999-11-18  32.546494  35.765381  28.612303  31.473534  27.068665   
A      1999-11-19  30.713520  30.758226  28.478184  28.880543  24.838577   
A      1999-11-22  29.551144  31.473534  28.657009  31.473534  27.068665   
A      1999-11-23  30.400572  31.205294  28.612303  28.612303  24.607880   
A      1999-11-24  28.701717  29.998211  28.612303  29.372318  25.261524   

            Volume source  Year  Close_lag_1  Adj_Close_lag_1   Returns  \
ticker                                                                    
A       62546300.0  A.csv  1999          NaN              NaN       NaN   
A       15234100.0  A.csv  1999    31.473534        27.068665 -0.082386   
A        6577800.0  A.csv  1999    28.880543        24.838577  0.089783   
A        5975600.0  A.csv  1999    31.473534        27.068665 -0.090909   
A        4843200.0  A.csv  1999    28.612303        24.607880  0.026563   

        Hi_Lo_range  
ticker               
A          7.153078  
A          2.280043  
A          2.816525  
A          2.592991  
A          1.385908

In [22]:
df_feat = df_feat.sort_values(['ticker', 'Date']) # we sort it so the rolling calculation makes chronological sense
df_feat['Returns_MA_10'] = df_feat.groupby('ticker')['Returns'].transform(lambda x: x.rolling(10).mean()) # Calculate the 10-day moving average of Returns for each ticker

In [23]:
df_feat.head

<bound method NDFrame.head of              Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      1999-11-18  32.546494  35.765381  28.612303  31.473534  27.068665   
A      1999-11-19  30.713520  30.758226  28.478184  28.880543  24.838577   
A      1999-11-22  29.551144  31.473534  28.657009  31.473534  27.068665   
A      1999-11-23  30.400572  31.205294  28.612303  28.612303  24.607880   
A      1999-11-24  28.701717  29.998211  28.612303  29.372318  25.261524   
...           ...        ...        ...        ...        ...        ...   
ZEUS   2020-03-26   9.610000   9.940000   9.260000   9.590000   9.590000   
ZEUS   2020-03-27   9.330000   9.330000   8.700000   8.700000   8.700000   
ZEUS   2020-03-30   8.810000   9.760000   8.700000   9.680000   9.680000   
ZEUS   2020-03-31   9.640000  10.470000   9.590000  10.350000  10.350000   
ZEUS   2020-04-01  10.140000  10.340000   9.310000   9.470

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Given that the data is not too large, working with pandas is a good option.
What I found is that you can do this in Dask but is more complicated.
Dask only supports .rolling() when the window size is known and the DataFrame is appropriately partitioned and sorted.
Dask's rolling support is less flexible than Pandas (e.g., .transform() inside .groupby() is limited)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.